In [1]:
import wrapper
from smart_agent_env import TerraBotEnvironment
import torch
import torch.nn as nn
import torch.optim as optim

env = TerraBotEnvironment()

In [6]:
WEIGHTS_PATH = "/home/robotanist/Desktop/TerraBot/agents/smart_agent/weights.pt"

In [2]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Define the policy network
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, 32)
        self.fc2 = nn.Linear(32, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)

policy = Policy()

In [3]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(policy.parameters())

def update_policy(rewards, log_probs, optimizer):
    log_probs = torch.stack(log_probs)
    loss = -torch.mean(log_probs * sum(rewards))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
policy = torch.load(WEIGHTS_PATH)

In [4]:
for episode in range(1000):
	state, _ = env.reset()
	done = False
	rewards = []
	log_probs = []
	
	while not done:
		# Select action
		state = torch.tensor(state, dtype=torch.float32).reshape(1, -1)
		probs = policy(state)
		action = torch.multinomial(probs, 1).item()
		log_prob = torch.log(probs[0, action])

		# Take step
		next_state, reward, done = env.step(action)
		rewards.append(reward)
		log_probs.append(log_prob)
		state = next_state
		
	# Update policy
	print(f"Episode {episode}: {sum(rewards)}")
	update_policy(rewards, log_probs, optimizer)
	rewards = []
	log_probs = []


Episode 0: 1000000000.0
Episode 1: 1000000000.0
Episode 2: 1000000000.0
Episode 3: 1000000000.0
Episode 4: 1000000000.0
Episode 5: 1000000000.0
Episode 6: 1000000000.0
Episode 7: 1000000000.0
Episode 8: 1000000000.0
Episode 9: 1000000000.0
Episode 10: 1000000000.0
Episode 11: 1000000000.0
Episode 12: 1000000000.0
Episode 13: 1000000000.0
Episode 14: 1000000000.0
Episode 15: 1000000000.0
Episode 16: 1000000000.0
Episode 17: 1000000000.0
Episode 18: 1000000000.0
Episode 19: 1000000000.0
Episode 20: 1000000000.0
Episode 21: 1000000000.0
Episode 22: 1000000000.0
Episode 23: 1000000000.0
Episode 24: 1000000000.0
Episode 25: 1000000000.0
Episode 26: 1000000000.0
Episode 27: 1000000000.0
Episode 28: 1000000000.0
Episode 29: 1000000000.0
Episode 30: 1000000000.0
Episode 31: 1000000000.0
Episode 32: 1000000000.0
Episode 33: 1000000000.0
Episode 34: 1000000000.0
Episode 35: 1000000000.0
Episode 36: 1000000000.0
Episode 37: 1000000000.0
Episode 38: 1000000000.0
Episode 39: 1000000000.0
Episode 40

In [7]:
torch.save(policy.state_dict(), WEIGHTS_PATH)